# CNN 😎

#### Libraries

In [ ]:
import utilities as ff
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import *
from keras.models import *
from keras.losses import *
from keras.optimizers import *
from keras.utils import *


#### Dataset

In [ ]:
batch_size = 42
img_width, img_height = 128, 128
color_mode = "grayscale"
no_classes = 2
seed = 42


In [ ]:
train_rgb, test_rgb = ff.ready_to_be_used_dataset(
    batch_size,
    image_squared_size=img_width,
    color_mode="rgb",
)


In [ ]:
train_bw, test_bw = ff.ready_to_be_used_dataset(
    batch_size,
    image_squared_size=img_width,
    color_mode="grayscale",
)


In [ ]:
# image_batch, labels_batch = next(iter(train))
# print(image_batch.shape, labels_batch.shape)


#### RGB Model definition

https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(2))
model.summary()


In [ ]:
model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


#### Training

In [ ]:
history = model.fit(train_rgb, epochs=10, validation_data=test_rgb)

#### Performance evaluation

In [ ]:
def performance_plot(history):
    plt.figure(figsize=(16, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.ylabel("loss", size=12)
    plt.xlabel("epoch", size=12)
    plt.legend(["train", "val"], fontsize=10)

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])
    plt.ylabel("accuracy", size=12)
    plt.xlabel("epoch", size=12)
    plt.legend(["train", "val"], fontsize=10)

    plt.show()


performance_plot(history)
